<a href="https://colab.research.google.com/github/Computer-CGuy/etherpaisahipaisa/blob/master/JAX_TPU_%7C_Soil_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

In [2]:
import jax
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [3]:
import numpy as np

# We only need to import JAX's xla_client, not all of JAX.
from jax.lib import xla_client as xc
from jax.lib import xla_bridge as xb
xops = xc.ops

In [ ]:
xb._get_tpu_driver_backend("tpu")

In [5]:
times = 1

In [4]:
def timed(runer,args,rate=False,batch=1):
    global times
    import time
    t0 = time.time()
    # print(runer(args))
    ret = runer(args)
    # for x in range(1):
        
    #     args = runer(args)
    #     print(args)
    t1 = time.time()
    if(rate):
        print(f"{batch*times/((t1-t0)):,}")
        return ret
    print(f"{(t1-t0):,}s")
    return ret

In [ ]:
(np.random.randn(5)*batch).astype(dtype=np.uint64)

array([             6575798, 18446744073695788334,             16864771,
                    6378636,              3508388], dtype=uint64)

In [ ]:
xc.get_local_backend("cpu")

In [6]:

def run(c,batch=1,build=None):
    global dtype
    global compiled_computation
    global cpu_backend
    global ROUNDS
    # your notebook kernel to crash!
    # # get a cpu backend
    # cpu_backend = xc.get_local_backend("cpu")
    cpu_backend = xb._get_tpu_driver_backend("tpu")

    if(not build):
        computation = c.Build()
    else:
        computation = build
    
    # # compile graph based on shape
    compiled_computation = cpu_backend.compile(computation)

    # if(batch==1):
    #     As = np.array(1, dtype=dtype)
    # else:
        # As = np.random.randn(batch,dtype=unint64)
        # As = np.zeros(batch, dtype=np.uint64)
        # As = (np.random.randn(batch)*3000).astype(dtype=dtype)
    As = np.random.randint(0,0x7FFFFFFF-1,(25,batch),dtype=dtype)
    # print(As)
    state = [cpu_backend.buffer_from_pyval(np.int32(0)),cpu_backend.buffer_from_pyval(np.int32(ROUNDS))]
    print("Evalving")
    for x in range(25):
        state.append(cpu_backend.buffer_from_pyval(As[x]))
    print("Started")
    out = timed(compiled_computation.execute,state,rate=True,batch=batch)
    print(out[0].to_py())
    # timed(compiled_computation.execute,state,rate=True,batch=batch)
    # print(state)
    # device_out = compiled_computation.execute(state)
    # print(device_out)

In [ ]:
As = [np.random.randint(0,batch,batch,dtype=dtype) for x in range(25)]

In [ ]:
type(np.int32(0))

numpy.int32

In [4]:
from jax import device_put

x = np.random.normal(size=(10, 10)).astype(np.float32)
x = device_put(x)

In [ ]:
import jax.numpy as jnp
# jnp.ran
# jnp.randint(0,10000,(100,100))

AttributeError: ignored

In [7]:
def index(x,y):
    return (((x)%5)+5*((y)%5))

In [8]:
def update(l):
    l[0]=1
    return l

In [9]:
def listRoll(a):
    ret = []
    for i in range(5):
        x = a[(i+1)%5]
        ret.append(xops.Not(xops.Or(xops.Div(x,Template),xops.Mul(x,Template))))
    return ret

In [10]:
def push(l,p):
    n = len(l)
    if(p==1):
        temp = l[0]
        for x in range(n):
            l[x] = l[(x+1)%n]
        l[n-1] = temp

In [11]:
def rearrange(l,a):
    n = len(l)
    temp = [0 for x in range(n)]
    for x in range(n):
        temp[x] = l[x]
    for x in range(n):
        state[x] = temp[a[x]]

In [12]:
def ROL64(a,offset):
    return xops.Xor(xops.Mul(a,offset[0]),xops.Div(a,offset[0]))
def rhoROL64(a):
    for i in range(len(a)):
        a[i] = ROL64(a[i],[Template,Template])

In [13]:
dtype = np.int32

In [14]:
miner = xc.XlaBuilder("Main Function")
batch=1000000
print(f"{batch:,}")
# batch=1
# Constants
KeccakRoundConstants = [xops.Constant(miner, dtype(0x000000000000800A))]
Template = xops.Constant(miner, dtype(12312123213))

pi_shifter = [0, 6, 12, 18, 24, 3, 9, 10, 16, 22, 1, 7, 13, 19, 20, 4, 5, 11, 17, 23, 2, 8, 14, 15, 21]

param_shape = xc.Shape.array_shape(np.dtype(dtype),(batch,))
param_shape2 = xc.Shape.array_shape(np.dtype(dtype),())

state = [param_shape2,param_shape2]+[xc.Shape.array_shape(np.dtype(dtype),(batch,)) for x in range(25)]

in_tuple_shape = xc.Shape.tuple_shape(state)
# Custom Constants
tup = xops.Parameter(miner,0,in_tuple_shape)

HASHES = xops.GetTupleElement(tup,0)

ROUND = xops.GetTupleElement(tup,1)
ROUND = xops.Sub(ROUND,xops.Constant(miner, np.int32(1)))

state = []
for x in range(25):
    state.append(xops.GetTupleElement(tup,x+2))

C = [0 for x in range(5)]

# Theta Function
for x in range(5):
    C[x] = state[index(x,0)]
    for y in range(1,5):
        C[x]=xops.Xor(state[index(x,y)],C[x])
D = listRoll(C)
# push(C,1)
for x in range(5):
    for y in range(5):
        state[index(x,y)]=xops.Xor(state[index(x,y)],xops.Xor(D[(x+1)%5],C[(x+4)%5]))
# print(state)

# Rho Function
rhoROL64(state)
# print(state)
# Pi Function

rearrange(state,pi_shifter)
# print(state)
# Chi Function

for y in range(5):
    for x in range(5):
        C[x] = xops.And(xops.Xor(state[index(x, y)],xops.Not((state[index(x+1, y)]))),state[index(x+2, y)])
    for x in range(5):
        state[index(x,y)] = C[x]
"""# print(state)
# Iota
state[0]=xops.Xor(state[0],KeccakRoundConstants[0])
# print(state)

# Output
"""
state[0]=xops.Xor(state[0],KeccakRoundConstants[0])
# out = xops.Tuple(miner,[ROUND,xops.Tuple(miner,state)])
xops.Tuple(miner,[HASHES,ROUND]+state)
body_computation = miner.build()
# run(miner,batch=batch)



1,000,000


In [82]:
1000000*500000

500000000000

In [16]:
ROUNDS = 24

In [15]:
# test computation:
tcb = xc.XlaBuilder("testcomp")
tup = xops.Parameter(tcb, 0, in_tuple_shape)
x = xops.GetTupleElement(tup,1)

const0 = xops.Constant(tcb, np.int32(0))
y = xops.Gt(x, const0)
test_computation = tcb.Build()

In [17]:
wcb = xc.XlaBuilder("whilecomp")
hashes = xops.Parameter(wcb, 0, param_shape2)
counter = xops.Parameter(wcb, 1, param_shape2)
records = [hashes,counter]
for x in range(25):
    records.append(xops.Parameter(wcb, x+2, param_shape))

# mixedTuple = xc.Shape.Tuple(counter,xc.Shape.tuple_shape(x))
res = xops.Tuple(wcb,records)
# res = xops.Tuple(wcb,[counter,b])
xops.While(test_computation, body_computation, res)
wc = wcb.build()

In [18]:
run(wcb,batch=batch,build=wc)
# Rounds 24
# Single Run

Evalving
Started
1,055,138.9643583086
0


In [19]:
# test computation:
tce = xc.XlaBuilder("processalgo")
tup = xops.Parameter(tce, 0, in_tuple_shape)
x = xops.GetTupleElement(tup,0)
x = xops.Add(x,xops.Constant(tce, np.int32(1)))
states = [x]
for x in range(26):
    states.append(xops.GetTupleElement(tup,x+1))
res = xops.Call(tce,wc,states)
# res = xops.Tuple(tce,states)
processalgo = tce.Build()

In [99]:
0x7FFFFFFF

2147483647

In [100]:
50000000000

50000000000

In [20]:
times = 100000
# test computation:
tcb = xc.XlaBuilder("testalgo")
tup = xops.Parameter(tcb, 0, in_tuple_shape)
x = xops.GetTupleElement(tup,0)

const0 = xops.Constant(tcb, np.int32(times))
y = xops.Gt(const0, x)
testalgo = tcb.Build()

In [21]:
wcb = xc.XlaBuilder("whilealgo")
hashes = xops.Parameter(wcb, 0, param_shape2)
counter = xops.Parameter(wcb, 1, param_shape2)
records = [hashes,counter]
for x in range(25):
    records.append(xops.Parameter(wcb, x+2, param_shape))
# mixedTuple = xc.Shape.Tuple(counter,xc.Shape.tuple_shape(x))
res = xops.Tuple(wcb,records)
# res = xops.Tuple(wcb,[counter,b])
xops.While(testalgo, processalgo, res)
wce = wcb.build()

In [22]:
run(wcb,batch=batch,build=wce)

# Several Rounds at a time Run

Evalving
Started
95,621,953,610.52722
100000


In [89]:
(500,000,000,000),(512,274,772,381)
659000000
2,147,483,647

659000000

In [21]:
miner = xc.XlaBuilder("Main Function")
batch=100000000
batch=100000
# Constants
KeccakRoundConstants = [xops.Constant(miner, dtype(0x000000000000800A))]
Template = xops.Constant(miner, dtype(12312123213))

pi_shifter = [0, 6, 12, 18, 24, 3, 9, 10, 16, 22, 1, 7, 13, 19, 20, 4, 5, 11, 17, 23, 2, 8, 14, 15, 21]

param_shape = xc.Shape.array_shape(np.dtype(dtype),(batch,))

state = []
for x in range(25):
    state.append(xops.Parameter(miner,x,param_shape))

# Custom Constants
C = [0 for x in range(5)]


# Theta Function
for x in range(5):
    C[x] = state[index(x,0)]
    for y in range(1,5):
        C[x]=xops.Xor(state[index(x,y)],C[x])
D = listRoll(C)
# push(C,1)
for x in range(5):
    for y in range(5):
        state[index(x,y)]=xops.Xor(state[index(x,y)],xops.Xor(D[(x+1)%5],C[(x+4)%5]))
# print(state)

# Rho Function
rhoROL64(state)
# print(state)
# Pi Function
rearrange(state,pi_shifter)
# print(state)
# Chi Function

for y in range(5):
    for x in range(5):
        C[x] = xops.And(xops.Xor(state[index(x, y)],xops.Not((state[index(x+1, y)]))),state[index(x+2, y)])
    for x in range(5):
        state[index(x,y)] = C[x]
# print(state)
# Iota
state[0]=xops.Xor(state[0],KeccakRoundConstants[0])
# print(state)

# Output
out = xops.Tuple(miner,state)

run(miner,batch=batch)

Evalving
Started
2,243,727,995,928,439.5
[-1776246494   270532736 -1030676150 ...  1761780038   814486345
   178323546]


In [ ]:
run(miner,batch=batch)

[DeviceArray(0, dtype=uint64), DeviceArray(1, dtype=uint64), DeviceArray(1, dtype=uint64)]


array(0, dtype=uint64)

In [ ]:
Xsqr = 2

guess = 1.0
converged_delta = 0.001
maxit = 1000

in_shape_0 = xc.Shape.array_shape(np.dtype(np.float32), ())
in_shape_1 = xc.Shape.array_shape(np.dtype(np.float32), ())
in_shape_2 = xc.Shape.array_shape(np.dtype(np.int32), ())
in_tuple_shape = xc.Shape.tuple_shape([in_shape_0, in_shape_1, in_shape_2])

# body computation:
# x_{i+1} = x_i - (x_i**2 - y) / (2 * x_i)
bcb = xc.XlaBuilder("bodycomp")
intuple = xops.Parameter(bcb, 0, in_tuple_shape)
y = xops.GetTupleElement(intuple, 0)
x = xops.GetTupleElement(intuple, 1)
guard_cntr = xops.GetTupleElement(intuple, 2)
new_x = xops.Sub(x, xops.Div(xops.Sub(xops.Mul(x, x), y), xops.Add(x, x)))
result = xops.Tuple(bcb, [y, new_x, xops.Sub(guard_cntr, xops.Constant(bcb, np.int32(1)))])
body_computation = bcb.Build()

# test computation -- convergence and max iteration test
tcb = xc.XlaBuilder("testcomp")
intuple = xops.Parameter(tcb, 0, in_tuple_shape)
y = xops.GetTupleElement(intuple, 0)
x = xops.GetTupleElement(intuple, 1)
guard_cntr = xops.GetTupleElement(intuple, 2)
criterion = xops.Abs(xops.Sub(xops.Mul(x, x), y))
# stop at convergence criteria or too many iterations
test = xops.And(xops.Gt(criterion, xops.Constant(tcb, np.float32(converged_delta))), 
               xops.Gt(guard_cntr, xops.Constant(tcb, np.int32(0))))
test_computation = tcb.Build()

# while computation:
# since jax does not allow users to create a tuple input directly, we need to
# take multiple parameters and make a intermediate tuple before feeding it as
# an initial carry to while loop
wcb = xc.XlaBuilder("whilecomp")
y = xops.Parameter(wcb, 0, in_shape_0)
x = xops.Parameter(wcb, 1, in_shape_1)
guard_cntr = xops.Parameter(wcb, 2, in_shape_2)
tuple_init_carry = xops.Tuple(wcb, [y, x, guard_cntr])
xops.While(test_computation, body_computation, tuple_init_carry)
while_computation = wcb.Build()

# Now compile and execute:
cpu_backend = xc.get_local_backend("cpu")

# compile graph based on shape
compiled_computation = cpu_backend.compile(while_computation)

y = np.array(Xsqr, dtype=np.float32)
x = np.array(guess, dtype=np.float32)
maxit = np.array(maxit, dtype=np.int32)

device_input_y = cpu_backend.buffer_from_pyval(y)
device_input_x = cpu_backend.buffer_from_pyval(x)
device_input_maxit = cpu_backend.buffer_from_pyval(maxit)
device_out = compiled_computation.execute([device_input_y, device_input_x, device_input_maxit])

# retrive the result
print(device_out)
print("square root of {y} is {x}".format(y=y, x=device_out[1].to_py()))

[DeviceArray(2., dtype=float32), DeviceArray(1.4142157, dtype=float32), DeviceArray(997, dtype=int32)]
square root of 2.0 is 1.4142156839370728
